In [ ]:
# graph 문제를 풀기 위한 torch, compuet vision문제를 보다 쉽게 해결할 수 있는 모듈인 torchvision import
import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Compose
import torch.nn.functional as F

In [ ]:
def set_manual_seed(seed=42):
  """
    난수 발생시 고정된 난수를 발생시키기 위해 python local package
    random, numpy (다차원의 메트릭을 다루기위환 package), pytorch
    seed를 고정하는 function 입니다.
  """
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
set_manual_seed()

In [ ]:
batch_size = 128
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('data', 
                   train=True, 
                   download=True,
                   transform=Compose([ToTensor()])),
                   batch_size=batch_size, 
                   shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('data', 
                   train=False, 
                   download=True,
                   transform=Compose([ToTensor()])),
                   batch_size=batch_size, 
                   shuffle=True)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
"""
  hyper parameters
"""
num_epochs = 2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr = 1e-3

In [ ]:
"""
  Operation : 
  -Convolution 2D(width, height) => 이미지의 특징을 보다 잘 찾아내기 위해
  -MaxPooling 2D(width, geifht) =>잘 찾아진 특징을 풀링을 통해서 가장 큰 값만 얻어내기 위한 기법
  -ReLu(Activation function) =>비선형 함수
  -Dropout(random drop out => features) 오버핏팅을 피하기 위한 기법
  위의 기법들을 반복하는것이 딥러닝이다. 
"""

class ConvoltuonNeuralNet(nn.Module):
    def __init__(self):
        super(ConvoltuonNeuralNet, self).__init__()

        self.feature_extraction = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=5),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Conv2d(10, 20, kernel_size=5),
            nn.Dropout(),
            nn.MaxPool2d(2),
            nn.ReLU(),
        )
        self.fully_connected = nn.Sequential(
            nn.Linear(320, 50),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(50, 10),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.feature_extraction(x)
        # print(f'feature extraction size: {x.size()}')
        x = x.view(-1, 320)
        # print(f'transfrom size: {x.size()}')
        x = self.fully_connected(x)

        return x

In [ ]:
model = ConvoltuonNeuralNet().to(device)

In [ ]:
optim = torch.optim.SGD(model.parameters(),lr=lr,momentum=0.5)

In [ ]:
def train_op(model, device, train_loader, optim, epoch):
    model.train()
    for batch_idx, (input_train_images, input_train_labels) in enumerate(train_loader):
        input_train_images, input_train_labels = input_train_images.to(device), input_train_labels.to(device)

        optim.zero_grad()
        """
          y: 부추라고 생각해
        """
        y = model(input_train_images)
        """
            정답과 예측한 값을 비교한뒤 맞으면 0 틀리면 1
        """
        loss = F.nll_loss(y.log(), input_train_labels)
        """
            결과를 다시 전달해준다
        """
        loss.backward()
        optim.step()
        if batch_idx % 100 == 0:
            print(f'Convoltuion Loss: {loss.item():.4f}')
            

# def test_op(model, device, test_loader):
#     pass

In [ ]:
for epoch in range(num_epochs):
    train_op(model=model, device=device, train_loader=train_loader, optim=optim, epoch=epoch)

Convoltuion Loss: 2.1495
Convoltuion Loss: 2.0573
Convoltuion Loss: 1.9489
Convoltuion Loss: 1.8906
Convoltuion Loss: 1.7204
Convoltuion Loss: 1.6517
Convoltuion Loss: 1.6156
Convoltuion Loss: 1.5395
Convoltuion Loss: 1.4618
Convoltuion Loss: 1.3916
